In [1]:
import json
import requests
import pandas as pd

host = "https://1e2d0164-f620-4188-849f-f2bd90b2ba46.deepnoteproject.com"


In [2]:
#Retrieve and print the list of monitored devices.
response = requests.get(host + '/devices')
response_addresses = json.loads(response.content.decode())
print(response_addresses)

{'mac_addresses': ['0x8ecce21a8aee', '0xbb7de14c93be']}


In [5]:
#For each device, retrieve the battery status from a given time range. Plot the battery level over time on a line chart
battery_status = pd.DataFrame()

for address in response_addresses['mac_addresses']:
    response = requests.get(host + "/device" + f"/{address}"  + f"?from=1674552201601&to=1674552205640")

    if response.ok:
        response_dict = json.loads(response.content.decode())

        for timestamp, battery_level, powerplugged in zip(response_dict["timestamps"], 
                                                        response_dict["battery_levels"],
                                                        response_dict["power_plugged"] ):
            output_dict = {
                "mac_address" : address,
                "timestamps": timestamp, 
                "battery_levels" : battery_level,
                "power_plugged" : powerplugged
            }

            battery_status = battery_status.append(output_dict, ignore_index=True)

    else:
        print(response.text)

In [6]:
DeepnoteChart(battery_status, """{"layer":[{"mark":{"clip":true,"type":"trail","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"timestamps","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"battery_levels","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"mac_address","scale":{"scheme":"set1"}}}},{"mark":{"clip":true,"type":"trail","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"timestamps","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"power_plugged","scale":{"type":"linear","zero":false}},"color":{"sort":null,"type":"nominal","field":"mac_address","scale":{"scheme":"tableau10"}}}}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [7]:
#Get the last device
response = requests.get(host + '/devices')
response_dict = json.loads(response.content.decode())
last_mac = response_dict['mac_addresses'][-1]
print(last_mac)

0xbb7de14c93be


In [ ]:
#delete the time series associated with the last MAC address
response = requests.delete(host + f'/device/{last_mac}')
print(response.text)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e2d0164-f620-4188-849f-f2bd90b2ba46' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>